In [7]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from matplotlib import pyplot as plt
import matplotlib.style as style

In [12]:
style.use('fivethirtyeight')
imputed = 'PATIENT_DATA_ALL.csv'
non_imputed = 'filename_2.txt'

In [24]:
#merged = np.genfromtxt(imputed, delimiter='\t')
merged = pd.read_csv(imputed, sep=',', )
labels = pd.read_csv('OLD_DATA/data_clinical_sample_clean.tsv',sep='\t')

In [25]:
labels.set_index('SAMPLE_ID', inplace=True)
labels.drop(['Unnamed: 0'], axis=1,inplace=True)

In [28]:
merged.drop('PATIENT_ID', axis=1,inplace=True)

In [30]:
merged = merged.transpose()

In [31]:
merged

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
SAMPLE_ID,MBC-MBCProject_1ps2hZH8-Tumor-SM-DL4OO,MBC-MBCProject_1qhlIasw-Tumor-SM-AXGNH,MBC-MBCProject_27uAugT4-Tumor-SM-DL45T,MBC-MBCProject_3jhES9fq-Tumor-SM-AXGIU,MBC-MBCProject_3LI1TNs3-Tumor-SM-DL43B,MBC-MBCProject_3MUnT7Ty-Tumor-SM-CGLLE,MBC-MBCProject_4DIpSBFV-Tumor-SM-CGM2Q,MBC-MBCProject_4MF1FlFQ-Tumor-SM-CGM4M,MBC-MBCProject_4MF1FlFQ-Tumor-SM-AZ5CU,MBC-MBCProject_4OIOfAt9-Tumor-SM-CGLYL,...,MBC-MBCProject_wAiri7fp-Tumor-SM-AZ5E4,MBC-MBCProject_wAiri7fp-Tumor-SM-AZ5DH,MBC-MBCProject_wKuZuQS1-Tumor-SM-CGLW3,MBC-MBCProject_xBfJfri9-Tumor-SM-CGLAP,MBC-MBCProject_xBfJfri9-Tumor-SM-CGL4F,MBC-MBCProject_Y7fYC1iG-Tumor-SM-CGLFQ,MBC-MBCProject_ZdudUNFZ-Tumor-SM-CGLPS,MBC-MBCProject_ZPHRs3Uw-Tumor-SM-DL3BA,MBC-MBCProject_zyt5TKFB-Tumor-SM-CGM59,MBC-MBCProject_zyt5TKFB-Tumor-SM-CGMBJ
ABEMACICLIB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANASTROZOLE,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
BEVACIZUMAB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
CAPECITABINE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CARBOPLATIN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CISPLATIN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CYCLOPHOSPHAMIDE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
DENOSUMAB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DOCETAXEL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
